# TIENDA ONLINE - ANÁLISIS DE MEJORAS/CAMBIOS (TEST A|B)

## INICIALIZACIÓN 

In [ ]:
# Cargar todas las librerías

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as st
import calendar
from datetime import datetime
import plotly.express as px
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from plotly import graph_objects as go
from statsmodels.stats.proportion import proportions_ztest

## CARGA DE DATOS

In [ ]:
url_1 = 'https://raw.githubusercontent.com/doalpinc98/online_store_ab_test/main/ab_project_marketing_events_us.csv'
data_1 = pd.read_csv(url_1, sep=';')

In [ ]:
url_2 = 'https://raw.githubusercontent.com/doalpinc98/online_store_ab_test/main/final_ab_new_users_upd_us.csv'
data_2 = pd.read_csv(url_2, sep=',')

In [ ]:
url_3 = 'https://raw.githubusercontent.com/doalpinc98/online_store_ab_test/main/final_ab_events_upd_us.csv'
data_3 = pd.read_csv(url_3, sep=',')

In [ ]:
url_4 = 'https://raw.githubusercontent.com/doalpinc98/online_store_ab_test/main/final_ab_participants_upd_us.csv'
data_4 = pd.read_csv(url_4, sep=',')

## PROCESAMIENTO Y EXPLORACIÓN DE DATOS

In [ ]:
data_1.info()


ab_project_marketing_events_us.csv: el calendario de eventos de marketing para 2020 <br>
Estructura `ab_project_marketing_events_us.csv`:

- `name`: el nombre del evento de marketing
- `regions`: regiones donde se llevará a cabo la campaña publicitaria
- `start_dt`: fecha de inicio de la campaña
- `finish_dt`: fecha de finalización de la campaña

In [ ]:
data_1.head()

In [ ]:
data_1['start_dt'] = pd.to_datetime(data_1['start_dt'], format='%Y-%m-%d')
data_1['finish_dt'] = pd.to_datetime(data_1['finish_dt'], format='%Y-%m-%d')

In [ ]:
data_1.info()

In [ ]:
data_2.info()

final_ab_new_users_upd_us.csv: todos los usuarios que se registraron en la tienda en línea desde el 7 hasta el 21 de diciembre de 2020 <br>
Estructura `final_ab_new_users_upd_us.csv`:

- `user_id`
- `first_date`: fecha de inscripción
- `region`
- `device`: dispositivo utilizado para la inscripción

In [ ]:
data_2['first_date'] = pd.to_datetime(data_2['first_date'], format='%Y-%m-%d')


In [ ]:
data_2.head()

In [ ]:
data_2.duplicated().sum()

In [ ]:
100*data_2.isna().sum()/data_2.shape[0]

In [ ]:
data_3.info()

In [ ]:
data_3 = data_3.drop(['details'], axis =1)

final_ab_events_upd_us.csv: todos los eventos de los nuevos usuarios en el período comprendido entre el 7 de diciembre de 2020 y el 1 de enero de 2021 <br>
Estructura `final_ab_events_upd_us.csv`:

- `user_id`
- `event_dt`: fecha y hora del evento
- `event_name`: nombre del tipo de evento
- `details`: datos adicionales sobre el evento (por ejemplo, el pedido total en USD para los eventos `purchase`)

In [ ]:
100*data_3.isna().sum()/data_3.shape[0]

In [ ]:
data_3['event_name'].unique()

In [ ]:
event_count = data_3.groupby('event_name')[['user_id']].count().reset_index()
event_count.columns = ['event_name', 'frequency']
event_count = event_count.sort_values(by = 'frequency', ascending = False)
event_count

In [ ]:
100*data_3.isna().sum()/data_3.shape[0]

In [ ]:
data_3.duplicated().sum()

In [ ]:
data_3 = data_3.dropna()

In [ ]:
data_3.isna().sum()

In [ ]:
data_3['event_dt'] = data_3['event_dt'].astype('str')
data_3['event_dt'] = pd.to_datetime(data_3['event_dt'], format='%Y-%m-%d %H:%M:%S')


In [ ]:
data_3.head()

In [ ]:
data_3.info()

In [ ]:
data_4.info()

final_ab_participants_upd_us.csv: tabla con los datos de los participantes de la prueba <br>

Estructura `final_ab_participants_upd_us.csv`:

- `user_id`
- `ab_test`: nombre de la prueba
- `group`: el grupo de prueba al que pertenecía el usuario

In [ ]:
data_4.head()

In [ ]:
data_4.duplicated().sum()

## ANALISIS DE DATOS

•	¿Cuántos eventos hay en los registros?<br>

In [ ]:
data_3['event_name'].count()

•	¿Cuántos usuarios y usuarias hay en los registros?<br>

In [ ]:
len(data_3['user_id'].unique())

•	¿Cuál es el promedio de eventos por usuario?<br>

In [ ]:
data_3.shape[0]/len(data_3['user_id'].unique())

•	¿Qué periodo de tiempo cubren los datos? Encuentra la fecha máxima y mínima. Traza un histograma por fecha y hora. ¿Puedes tener seguridad de que tienes datos igualmente completos para todo el periodo? Los eventos más antiguos podrían terminar en los registros de algunos usuarios o usuarias por razones técnicas y esto podría sesgar el panorama general. Encuentra el momento en el que los datos comienzan a estar completos e ignora la sección anterior. ¿Qué periodo representan realmente los datos?<br>


In [ ]:
df = px.data.tips()
fig = px.histogram(data_3, x="event_dt", nbins=20)
fig.show()

•	Observa qué eventos hay en los registros y su frecuencia de suceso. Ordénalos por frecuencia.<br>

In [ ]:
event_count = data_3.groupby('event_name')[['user_id']].count().reset_index()
event_count.columns = ['event_name', 'frequency']
event_count = event_count.sort_values(by = 'frequency', ascending = False)
event_count

•	Encuentra la cantidad de usuarios y usuarias que realizaron cada una de estas acciones. Ordena los eventos por el número de usuarios y usuarias. Calcula la proporción de usuarios y usuarias que realizaron la acción al menos una vez.<br>

In [ ]:
event_users = data_3.groupby('event_name')[['user_id']].nunique().reset_index()
event_users.columns = ['event_name', 'user_id']
event_users = event_users.sort_values(by = 'user_id', ascending = False)
event_users

In [ ]:
fig = plt.figure(figsize = (16, 8))
 

plt.bar(event_users['event_name'], event_users['user_id'] )
 
plt.xlabel("event_name")
plt.ylabel("user_id")
plt.title("Number of Users by event")
plt.show()

•	Utiliza el embudo de eventos para encontrar la proporción de usuarios y usuarias que pasan de una etapa a la siguiente. (Por ejemplo, para la secuencia de eventos A → B → C, calcula la proporción de usuarios en la etapa B a la cantidad de usuarios en la etapa A y la proporción de usuarios en la etapa C a la cantidad en la etapa B).<br>

In [ ]:
funnel = event_users
funnel

In [ ]:
fig = go.Figure(go.Funnel(
    y = funnel['event_name'],
    x = funnel['user_id'],
    textposition = "inside",
    textinfo = "value+percent initial",
    opacity = 0.65, marker = {"color": ["deepskyblue", "lightsalmon", "tan", "teal"],
    "line": {"width": [4, 2, 2, 3, 1], "color": ["wheat", "wheat", "blue", "wheat"]}},
    connector = {"line": {"color": "royalblue", "dash": "dot", "width": 3}})
    )

fig.show()

In [ ]:
print(data_4.groupby('group')['user_id'].nunique())

In [ ]:
both = data_4.merge(data_3, on='user_id')

In [ ]:
both.head()

In [ ]:
groupA = both[both['group'] == 'A'].groupby('event_name')[['user_id']].nunique().reset_index()
groupA.columns = ['event_name', 'users']

groupA = groupA[groupA['event_name'] != 'Tutorial']
groupA= groupA.sort_values(by = 'users', ascending = False)

In [ ]:
groupA.shape

In [ ]:
groupA

In [ ]:
groupB = both[both['group'] == 'B'].groupby('event_name')[['user_id']].nunique().reset_index()
groupB.columns = ['event_name', 'users']

groupB = groupB[groupB['event_name'] != 'Tutorial']
groupB= groupB.sort_values(by = 'users', ascending = False)

In [ ]:
groupB

In [ ]:
def z_test (exito_a, tamaño_a, exito_b, tamaño_b):
    
#tamaño de A: 7874
#tamaño de B  6205  
#0,05/4 = 0,0125
    alpha = 0.0125


    successes = np.array([exito_a, exito_b])
    samples = np.array([tamaño_a, tamaño_b])

    
    stat, p_value = proportions_ztest(count=successes, nobs=samples,  alternative='two-sided')

    
    print('z_stat: %0.3f, p_value: %0.3f' % (stat, p_value))
    if p_value < alpha:
       print ("Se rechaza la hipótesis nula")
    else:
       print ("No se rechaza la hipótesis nula")

Product page vs Usuarios totales

In [ ]:
z_test (5208, 7874, 3986, 6205)

login vs product page

In [ ]:
z_test (4035, 5208, 3319, 3986)

Product cart vs login

In [ ]:
z_test (2682,4035, 2008, 3319)

## CONCLUSIÓN